In [60]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [62]:
from typing import Tuple

import pandas as pd

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor

### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [63]:
ds = Dataset('../data/dataset/')

In [64]:
long = ds['daily-min-temperatures.csv']

In [65]:
plot_ts(long)

Loading BokehJS ...

In [66]:
short_season = ds['hour_3019.csv'][300:]

In [67]:
plot_ts(short_season)

Loading BokehJS ...

In [68]:
short_season_trend = ds['international-airline-passengers.csv']

In [69]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [70]:
random_walk = ds['dow_jones_0.csv']

In [71]:
plot_ts(random_walk)

Loading BokehJS ...

In [72]:
medium_noize = ds['hour_3426.csv'][300:]

In [73]:
plot_ts(medium_noize)

Loading BokehJS ...

In [74]:
full_noize = ds['day_1574.csv']

In [75]:
plot_ts(full_noize)

Loading BokehJS ...

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Ridge)
* RandomForeset
* градиентный бустинг


In [76]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mape
* mase

In [77]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

from metrics import mean_absolute_percentage_error as mape
from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"
* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1. Напишите функцию, разбивающую на train и test

In [78]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    split_idx = int(len(ts) * ratio)
    ts_train, ts_test = ts[:split_idx], ts[split_idx:]
    # ваш код здесь
    return ts_train, ts_test

### Зададим соответствие гранулярностей для наших рядов.

In [79]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [80]:
def make_pipeline(ts: pd.Series, granularity: str, num_lags: int, model: callable, **kwargs) -> Tuple[pd.Series]: 
    train, test = train_test_split(ts)
    predictor = TimeSeriesPredictor(
        granularity=granularity,
        num_lags=num_lags,
        model=model,
        **kwargs
    )
    predictor.fit(train)
    
    in_sample = predictor.predict_batch(train, test)
    out_of_sample = predictor.predict_next(train, len(test))
    
    return train, test, in_sample, out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [81]:
def make_pipeline_arima(ts: pd.Series, period: int, d: int = 1, D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    train, test = train_test_split(ts)
    
    arima_fit = auto_arima(
        train,
        max_p=3, max_q=3, m=period,
        seasonal=seasonal,
        d=d, D=D,
        trace=True,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )
    
    out_of_sample = pd.Series(data=arima_fit.predict(len(test)), index=test.index)
    
    arima_fit.fit(ts)
    
    in_sample = pd.Series(arima_fit.predict_in_sample()[-len(test):], index=test.index)
    
    return train, test, in_sample, out_of_sample

### Задание 4. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 

In [84]:
plot_ts(*make_pipeline_arima(short_season_trend, period=12, seasonal=False, d=1, D=1))

/Users/gsinyakov/ts_spring/.env/lib/python3.8/site-packages/pmdarima/arima/_validation.py:62: UserWarning: m (12) set for non-seasonal fit. Setting to 0
  warnings.warn("m (%i) set for non-seasonal fit. Setting to 0" % m)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=909.784, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=907.453, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=905.340, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=908.808, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=900.705, Time=0.06 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.13 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.12 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=905.506, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=899.587, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=904.024, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=906.151, Time=0.02 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=897.042, Time=0.05 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=904.526, Time=0.02 se

Loading BokehJS ...

In [83]:
from itertools import product

param_grid = {
    'max_depth': [6, 12],
    'n_estimators': [50, 500, 1000],
    'num_lags': [7, 14]
}

for param_tuple in product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), param_tuple))
    num_lags = params.pop('num_lags')
    train, test, in_sample, out_of_sample = make_pipeline(
        full_noize, 'P1D', num_lags, RandomForestRegressor, **params)
    
    print(f'Params are: {params}, num_lags are: {num_lags}')
    print(
        f"""
        IN_SAMPLE
        mse: {mse(test, in_sample)},
        mae: {mae(test, in_sample)},
        r2: {r2_score(test, in_sample)},
        mase: {mase(in_sample, test)}
        ---------------------
        OUT_OF_SAMPLE
        mse: {mse(test, out_of_sample)},
        mae: {mae(test, out_of_sample)},
        r2: {r2_score(test, out_of_sample)},
        mase: {mase(out_of_sample, test)}
        """
    )
    plot_ts(train, test, in_sample, out_of_sample)

Params are: {'max_depth': 6, 'n_estimators': 50}, num_lags are: 7

        IN_SAMPLE
        mse: 4.4467405807371565e-05,
        mae: 0.005641377395813691,
        r2: -0.4013994670969274,
        mase: 0.8488545540137818
        ---------------------
        OUT_OF_SAMPLE
        mse: 3.452785914248513e-05,
        mae: 0.004971175285840857,
        r2: -0.08815260354709187,
        mase: 0.7480096586550253
        


Loading BokehJS ...

Params are: {'max_depth': 6, 'n_estimators': 50}, num_lags are: 14

        IN_SAMPLE
        mse: 4.264955833713664e-05,
        mae: 0.005053431368007137,
        r2: -0.34410962907295173,
        mase: 0.7603866802657392
        ---------------------
        OUT_OF_SAMPLE
        mse: 3.297506925956046e-05,
        mae: 0.004804440187061508,
        r2: -0.039216110065311005,
        mase: 0.7229211318676105
        


Loading BokehJS ...

KeyboardInterrupt: 